In [5]:
import numpy as np
import pandas as pd

import pymysql
import dotenv
import os
import pickle
import seaborn as sns
import matplotlib

from datetime import timedelta

import matplotlib.pyplot as plt

# Normalization/Standardization
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout, Conv1D, GRU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.saving import save_model

# 경고 무시 코드 추가
import warnings
warnings.filterwarnings('ignore')

def pltconfig_default() :
  sns.reset_defaults()
  %matplotlib inline

pltconfig_default()

matplotlib.rcParams

matplotlib.rcParams['font.family']

current_font_list = matplotlib.rcParams['font.family']

font_path = 'C:\\Windows\\Fonts\\batang.ttc'

kfont = matplotlib.font_manager.FontProperties(fname=font_path).get_name()

matplotlib.rcParams['font.family'] = [kfont] + current_font_list

In [6]:
St_NotEncode_data = pd.read_pickle("StandardScalar_final_data")

print(St_NotEncode_data.shape)

(17422, 9)


In [7]:
# Feature와 Label 분리하기
def Feature_Label(datafile) :
    X = datafile.iloc[:,:-1]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SNE_X, SNE_y = Feature_Label(St_NotEncode_data)
print(SNE_X.shape, SNE_y.shape)
SNE_X_train, SNE_X_test, SNE_y_train, SNE_y_test = train_test_split(SNE_X, SNE_y, test_size=0.2, random_state=10, shuffle=False)
print(SNE_X_train.shape, SNE_X_test.shape, SNE_y_train.shape, SNE_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 8) (17422,)
(13937, 8) (3485, 8) (13937,) (3485,)


In [8]:
# X 값 Window Dataset 구성
ds_x = tf.data.Dataset.from_tensor_slices(SNE_X_train).window(WINDOW_SIZE, stride=1, shift=1, drop_remainder=True)  # 여기서는 X값만 별도로 구성하므로 WINDOW_SIZE 값에 1을 더하지 않는다!
ds_x = ds_x.flat_map(lambda x : x.batch(WINDOW_SIZE))

In [9]:
# y 값 Window Dataset 구성
ds_y = tf.data.Dataset.from_tensor_slices(SNE_y_train[WINDOW_SIZE:])

In [10]:
train_data = tf.data.Dataset.zip((ds_x, ds_y)).batch(BATCH_SIZE)

In [11]:
for x, y in train_data.take(1) :
  print(x[:3])
  print()
  print(y[:3])

tf.Tensor(
[[[-0.11352868 -0.09386187 -0.6825146   0.09445245 -0.66581843
   -1.23511514 -0.21825161 -1.53935952]
  [-0.11352868 -0.9225242  -1.06038321  0.4051263  -0.66581843
   -1.23511514 -0.21825161 -1.57681737]
  [-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834]]

 [[-0.11352868 -0.9225242  -1.06038321  0.4051263  -0.66581843
   -1.23511514 -0.21825161 -1.57681737]
  [-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834]
  [-0.11352868 -1.12968978 -1.06038321  0.71580015 -0.66581843
   -1.23511514 -0.21825161 -1.65922462]]

 [[-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834]
  [-0.11352868 -1.12968978 -1.06038321  0.71580015 -0.66581843
   -1.23511514 -0.21825161 -1.65922462]
  [-0.11352868 -0.9225242  -1.06038321  0.80456411 -0.66581843
   -1.23511514 -0.21825161 -1.68919089]]], shape=(3, 3, 8), dtype=float64)

tf.Tensor([0. 0. 0.], 

In [12]:
pd.concat([SNE_X_train, SNE_y_train], axis=1).head(10)

,0,1,2,3,4,5,6,7,8
0,-0.113529,-0.093862,-0.682515,0.094452,-0.665818,-1.235115,-0.218252,-1.539360,0.0
1,-0.113529,-0.922524,-1.060383,0.405126,-0.665818,-1.235115,-0.218252,-1.576817,0.0
2,-0.113529,-1.026107,-1.060383,0.538272,-0.665818,-1.235115,-0.218252,-1.629258,0.0
3,-0.113529,-1.129690,-1.060383,0.715800,-0.665818,-1.235115,-0.218252,-1.659225,0.0
4,-0.113529,-0.922524,-1.060383,0.804564,-0.665818,-1.235115,-0.218252,-1.689191,0.0
5,-0.113529,-1.026107,-1.060383,0.937710,-0.665818,-1.235115,-0.218252,-1.719157,0.0
6,-0.113529,-0.922524,-1.060383,0.893328,-0.665818,-1.235115,-0.218252,-1.734140,0.0
7,-0.113529,-0.922524,-1.060383,0.982092,0.549517,-1.235115,-0.333181,-1.741632,6000.0
8,-0.113529,-1.129690,-1.060383,0.538272,1.764852,0.769233,-0.226741,-1.636750,16000.0
9,-0.113529,-0.922524,-1.060383,0.005688,1.764852,-0.984572,-0.322080,-1.329596,24000.0


In [13]:
# 위 내용을 함수 형태로 만들고 싶은 경우

def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [14]:
train_data = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SNE_X_test, SNE_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [15]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 8), (28,)


In [16]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [17]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [18]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [19]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [20]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('new_multi_stne_lstm_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('new_multi_stne_lstm_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('new_multi_stne_rnn_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('new_multi_stne_rnn_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('new_multi_stne_gru_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('new_multi_stne_gru_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [21]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500


    494/Unknown - 11s 6ms/step - loss: 7605.1152 - mse: 183235040.0000 - mae: 7605.4990
Epoch 1: val_loss improved from inf to 7598.87744, saving model to new_multi_stne_lstm_weight.h5
498/498 [==============================] - 18s 20ms/step - loss: 7617.2183 - mse: 183439664.0000 - mae: 7617.6030 - val_loss: 7598.8774 - val_mse: 182641072.0000 - val_mae: 7599.2959 - lr: 0.0010
Epoch 2/500
497/498 [============================>.] - ETA: 0s - loss: 7569.7305 - mse: 181352448.0000 - mae: 7570.1113
Epoch 2: val_loss improved from 7598.87744 to 7538.59814, saving model to new_multi_stne_lstm_weight.h5
498/498 [==============================] - 5s 11ms/step - loss: 7571.7114 - mse: 181442064.0000 - mae: 7572.0923 - val_loss: 7538.5981 - val_mse: 179930656.0000 - val_mae: 7539.0225 - lr: 0.0010
Epoch 3/500
497/498 [============================>.] - ETA: 0s - loss: 7492.8687 - mse: 178171328.0000 - mae: 7493.2354
Epoch 3: val_loss improved from 7538.59814 to 7438.63623, saving model to new_mu

In [22]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
    493/Unknown - 8s 4ms/step - loss: 7615.2432 - mse: 183515776.0000 - mae: 7615.6157
Epoch 1: val_loss improved from inf to 7591.89014, saving model to new_multi_stne_rnn_weight.h5
498/498 [==============================] - 11s 11ms/step - loss: 7615.6450 - mse: 183384240.0000 - mae: 7616.0190 - val_loss: 7591.8901 - val_mse: 182320800.0000 - val_mae: 7592.2910 - lr: 0.0010
Epoch 2/500
498/498 [==============================] - ETA: 0s - loss: 7552.4482 - mse: 180603328.0000 - mae: 7552.8271
Epoch 2: val_loss improved from 7591.89014 to 7497.54736, saving model to new_multi_stne_rnn_weight.h5
498/498 [==============================] - 4s 7ms/step - loss: 7552.4482 - mse: 180603328.0000 - mae: 7552.8271 - val_loss: 7497.5474 - val_mse: 178337136.0000 - val_mae: 7497.9712 - lr: 0.0010
Epoch 3/500
493/498 [============================>.] - ETA: 0s - loss: 7423.6577 - mse: 175292560.0000 - mae: 7424.0254
Epoch 3: val_loss improved from 7497.54736 to 7369.79639, saving model t

In [23]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500


    494/Unknown - 20s 7ms/step - loss: 7612.6875 - mse: 183379136.0000 - mae: 7613.0742
Epoch 1: val_loss improved from inf to 7598.63281, saving model to new_multi_stne_gru_weight.h5
498/498 [==============================] - 23s 12ms/step - loss: 7614.9053 - mse: 183349184.0000 - mae: 7615.2915 - val_loss: 7598.6328 - val_mse: 182193936.0000 - val_mae: 7599.0918 - lr: 0.0010
Epoch 2/500
494/498 [============================>.] - ETA: 0s - loss: 7550.7681 - mse: 180782800.0000 - mae: 7551.1670
Epoch 2: val_loss improved from 7598.63281 to 7492.27734, saving model to new_multi_stne_gru_weight.h5
498/498 [==============================] - 4s 9ms/step - loss: 7545.1113 - mse: 180416816.0000 - mae: 7545.5107 - val_loss: 7492.2773 - val_mse: 178108160.0000 - val_mae: 7492.6670 - lr: 0.0010
Epoch 3/500
498/498 [==============================] - ETA: 0s - loss: 7420.3193 - mse: 175340928.0000 - mae: 7420.6992
Epoch 3: val_loss improved from 7492.27734 to 7343.76270, saving model to new_multi

In [24]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 183.9151153564453
lstm_history Validation MSE: 985574.125
lstm_history Validation MAE: 184.2042694091797
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 173.6879425048828
lstm_history_ns Validation MSE: 980430.375
lstm_history_ns Validation MAE: 173.95684814453125
-------------rnn_history-------------
rnn_history Validation Loss: 404.8813781738281
rnn_history Validation MSE: 2240386.25
rnn_history Validation MAE: 405.1925354003906
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 376.8558654785156
rnn_history_ns Validation MSE: 2188098.5
rnn_history_ns Validation MAE: 377.1513671875
-------------gru_history-------------
gru_history Validation Loss: 255.6411895751953
gru_history Validation MSE: 1417487.625
gru_history Validation MAE: 255.94346618652344
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 253.27981567382812
gru_history_ns Validation M

In [25]:
lstm_save_path = f"./Models/lstm_model_stne_all"
rnn_save_path = f"./Models/rnn_model_stne_all"
gru_save_path = f"./Models/gru_model_stne_all"

save_model(lstm_model, lstm_save_path, overwrite=True)
save_model(rnn_model, rnn_save_path, overwrite=True)
save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_all\assets


INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_all\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_all\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_all\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_all\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_all\assets


In [26]:
St_NotEncode_Basic_data = pd.read_pickle("Basic_StandardScalar_final_data")
  
print(St_NotEncode_Basic_data.shape)

(52578, 9)


In [27]:
# Feature와 Label 분리하기
def Feature_Label(datafile) :
    X = datafile.iloc[:,:-1]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SB_X, SB_y = Feature_Label(St_NotEncode_Basic_data)
print(SNE_X.shape, SNE_y.shape)
SB_X_train, SB_X_test, SB_y_train, SB_y_test = train_test_split(SB_X, SB_y, test_size=0.2, random_state=10, shuffle=False)
print(SB_X_train.shape, SB_X_test.shape, SB_y_train.shape, SB_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 8) (17422,)
(42062, 8) (10516, 8) (42062,) (10516,)


In [28]:
def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [29]:
train_data = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SB_X_test, SB_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [30]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 8), (28,)


In [31]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [32]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [33]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [34]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [35]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('base_multi_stne_lstm_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('base_multi_stne_lstm_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('base_multi_stne_rnn_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('base_multi_stne_rnn_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('base_multi_stne_gru_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('base_multi_stne_gru_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [36]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500


   1500/Unknown - 23s 8ms/step - loss: 2.1142 - mse: 24.2912 - mae: 2.4798
Epoch 1: val_loss improved from inf to 2.34999, saving model to base_multi_stne_lstm_weight.h5
1503/1503 [==============================] - 27s 10ms/step - loss: 2.1137 - mse: 24.2819 - mae: 2.4792 - val_loss: 2.3500 - val_mse: 23.9735 - val_mae: 2.7418 - lr: 0.0010
Epoch 2/500
1498/1503 [============================>.] - ETA: 0s - loss: 1.3253 - mse: 9.5682 - mae: 1.6729
Epoch 2: val_loss improved from 2.34999 to 2.27630, saving model to base_multi_stne_lstm_weight.h5
1503/1503 [==============================] - 13s 9ms/step - loss: 1.3273 - mse: 9.5938 - mae: 1.6750 - val_loss: 2.2763 - val_mse: 23.3862 - val_mae: 2.6638 - lr: 0.0010
Epoch 3/500
1499/1503 [============================>.] - ETA: 0s - loss: 1.2791 - mse: 9.0818 - mae: 1.6228
Epoch 3: val_loss improved from 2.27630 to 2.13817, saving model to base_multi_stne_lstm_weight.h5
1503/1503 [==============================] - 14s 9ms/step - loss: 1.2800 -

In [37]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500


   1501/Unknown - 10s 4ms/step - loss: 1.8724 - mse: 18.6097 - mae: 2.2245
Epoch 1: val_loss improved from inf to 2.40618, saving model to base_multi_stne_rnn_weight.h5
1503/1503 [==============================] - 12s 5ms/step - loss: 1.8718 - mse: 18.6017 - mae: 2.2240 - val_loss: 2.4062 - val_mse: 25.4696 - val_mae: 2.8038 - lr: 0.0010
Epoch 2/500
1499/1503 [============================>.] - ETA: 0s - loss: 1.3524 - mse: 9.8439 - mae: 1.6954
Epoch 2: val_loss improved from 2.40618 to 2.32972, saving model to base_multi_stne_rnn_weight.h5
1503/1503 [==============================] - 6s 4ms/step - loss: 1.3564 - mse: 9.9211 - mae: 1.6995 - val_loss: 2.3297 - val_mse: 25.2026 - val_mae: 2.7162 - lr: 0.0010
Epoch 3/500
1487/1503 [============================>.] - ETA: 0s - loss: 1.3190 - mse: 9.4812 - mae: 1.6607
Epoch 3: val_loss improved from 2.32972 to 2.09875, saving model to base_multi_stne_rnn_weight.h5
1503/1503 [==============================] - 6s 4ms/step - loss: 1.3245 - mse: 

In [38]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500


   1502/Unknown - 21s 7ms/step - loss: 2.0903 - mse: 23.7119 - mae: 2.4495
Epoch 1: val_loss improved from inf to 2.21861, saving model to base_multi_stne_gru_weight.h5
1503/1503 [==============================] - 25s 10ms/step - loss: 2.0902 - mse: 23.7103 - mae: 2.4494 - val_loss: 2.2186 - val_mse: 22.2228 - val_mae: 2.6271 - lr: 0.0010
Epoch 2/500
1502/1503 [============================>.] - ETA: 0s - loss: 1.3204 - mse: 9.5623 - mae: 1.6652
Epoch 2: val_loss improved from 2.21861 to 2.16222, saving model to base_multi_stne_gru_weight.h5
1503/1503 [==============================] - 13s 9ms/step - loss: 1.3208 - mse: 9.5672 - mae: 1.6656 - val_loss: 2.1622 - val_mse: 21.4976 - val_mae: 2.5653 - lr: 0.0010
Epoch 3/500
1499/1503 [============================>.] - ETA: 0s - loss: 1.2582 - mse: 8.9619 - mae: 1.5956
Epoch 3: val_loss improved from 2.16222 to 1.82820, saving model to base_multi_stne_gru_weight.h5
1503/1503 [==============================] - 12s 8ms/step - loss: 1.2600 - ms

In [39]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 1.7570290565490723
lstm_history Validation MSE: 16.333913803100586
lstm_history Validation MAE: 2.128230094909668
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 1.8002194166183472
lstm_history_ns Validation MSE: 16.84504508972168
lstm_history_ns Validation MAE: 2.155298948287964
-------------rnn_history-------------
rnn_history Validation Loss: 1.8009889125823975
rnn_history Validation MSE: 16.961101531982422
rnn_history Validation MAE: 2.160658359527588
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 1.8552051782608032
rnn_history_ns Validation MSE: 17.514455795288086
rnn_history_ns Validation MAE: 2.2290329933166504
-------------gru_history-------------
gru_history Validation Loss: 1.7341647148132324
gru_history Validation MSE: 16.444591522216797
gru_history Validation MAE: 2.09588623046875
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 1

In [40]:
lstm_save_path = f"./Models/lstm_model_stne_base_all"
rnn_save_path = f"./Models/rnn_model_stne_base_all"
gru_save_path = f"./Models/gru_model_stne_base_all"

save_model(lstm_model, lstm_save_path, overwrite=True)
save_model(rnn_model, rnn_save_path, overwrite=True)
save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_base_all\assets


INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_base_all\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_base_all\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_base_all\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_base_all\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_base_all\assets
